### 라이브러리 준비

In [3]:
# !pip install langchain-pinecone

In [4]:
import os
import pandas as pd

# LangChain
from langchain_openai import OpenAIEmbeddings
from langchain_pinecone import PineconeVectorStore
from langchain_core.documents import Document

# Pinecone
from pinecone import Pinecone, ServerlessSpec

# env
from dotenv import load_dotenv
load_dotenv()

True

### 상수 설정

In [5]:
PC_API_KEY = os.getenv("PINECONE_API_KEY")
INDEX_NAME = "funeral-index"

### document 생성 함수

In [6]:
def create_doc(csv_path):
    df = pd.read_csv(csv_path)
    df = df.fillna("정보없음")
    
    # 검색용 텍스트: "지역: 서울특별시, 소재지: 종로구, 시설명: 서울대병원장례식장, 연락처: 02-000-0000" 형태로 합침
    documents = []
    for _, row in df.iterrows():
        # 검색에 사용될 텍스트
        page_content = f"지역: {row['지역']}, 소재지: {row['소재지']}, 시설명: {row['시설명']}, 연락처: {row['연락처']}"
        
        # 메타데이터 (필터링이나 출력에 사용)
        metadata = {
            "region": row['지역'],
            "location": row['소재지'],
            "name": row['시설명'],
            "phone": row['연락처']
        }
        
        documents.append(Document(page_content=page_content, metadata=metadata))

    print(f"총 {len(documents)}개 데이터 준비")
    return documents

### DB 적재 함수 (Pinecone 사용)

In [52]:
def create_db(documents):
    # Pinecone 초기화 및 인덱스 생성 (없으면 생성)
    pc = Pinecone(api_key=PC_API_KEY)
    existing_indexes = [index_info["name"] for index_info in pc.list_indexes()]

    if INDEX_NAME not in existing_indexes:
        pc.create_index(
            name=INDEX_NAME,
            dimension=1536, # OpenAI text-embedding-3-small 기준
            metric="cosine",
            spec=ServerlessSpec(cloud="aws", region="us-east-1")
        )
        print(f"{INDEX_NAME} 인덱스 생성")

    # 벡터 DB에 업로드
    embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
    
    vector_store = PineconeVectorStore.from_documents(
        documents=documents,
        embedding=embeddings,
        index_name=INDEX_NAME
    )
    print("데이터 적재 완료")

### 실행

In [8]:
file_path = ".\\data\\processed\\bongandang.csv"

documents = create_doc(file_path)

총 626개 데이터 준비


In [9]:
documents

[Document(metadata={'region': '경상남도', 'location': '김해시', 'name': '대법륜사 극락전', 'phone': '055-322-5457'}, page_content='지역: 경상남도, 소재지: 김해시, 시설명: 대법륜사 극락전, 연락처: 055-322-5457'),
 Document(metadata={'region': '전라남도', 'location': '보성군', 'name': '봉갑사 납골묘', 'phone': '061-853-9979'}, page_content='지역: 전라남도, 소재지: 보성군, 시설명: 봉갑사 납골묘, 연락처: 061-853-9979'),
 Document(metadata={'region': '서울특별시', 'location': '동작구', 'name': '달마사봉안당', 'phone': '02-822-8771'}, page_content='지역: 서울특별시, 소재지: 동작구, 시설명: 달마사봉안당, 연락처: 02-822-8771'),
 Document(metadata={'region': '충청남도', 'location': '금산군', 'name': '만인산 만인사 봉안당', 'phone': '041-751-3755'}, page_content='지역: 충청남도, 소재지: 금산군, 시설명: 만인산 만인사 봉안당, 연락처: 041-751-3755'),
 Document(metadata={'region': '경상남도', 'location': '창녕군', 'name': '창녕군추모공원', 'phone': '055-533-2719'}, page_content='지역: 경상남도, 소재지: 창녕군, 시설명: 창녕군추모공원, 연락처: 055-533-2719'),
 Document(metadata={'region': '전라남도', 'location': '함평군', 'name': '상상추모공원', 'phone': '061-322-6666'}, page_content='지역: 전라남도, 소재지: 함평군, 시설

In [ ]:
# create_db(documents)

데이터 적재 완료
